In [21]:
# ! pip install -r ./requirements.txt

## 数据模型准备

In [22]:
from torch.utils.data import DataLoader,random_split
from ldct.data import *
from ldct.utils import *
from ldct.loss import *
from ldct.net.generator import *
from tqdm import tqdm
import numpy as np
import cv2 as cv

In [23]:
fd_path = "../../LDCT_SLL_1000_200/test/fd"
ld_path = "../../LDCT_SLL_1000_200/test/qd/"
dataset = LDCT_Dataset(fd_path,ld_path,crop_size=(512,512))

batch_size = 1
num_workers = 1
data_loader = DataLoader(dataset,batch_size,num_workers=num_workers)


model_path = "./model/temp/"
device = torch.device("cuda:0")


## 评价指标

In [24]:
# # LDCT
# psnr_list = []
# ssim_list = []
# gmsd_list = []
# rmse_list = []
# for img_ld,img_fd in tqdm(data_loader):
#     psnr_list.append(psnr(img_ld*255,img_fd*255).item())
#     ssim_list.append(ssim(img_ld*255,img_fd*255).item())
#     gmsd_list.append(gmsd(img_ld*255,img_fd*255).item())
#     rmse_list.append(rmse(img_ld*255,img_fd*255).item())
# print(f"{np.mean(psnr_list):.3f}\t{np.mean(ssim_list):.3f}\t{np.mean(gmsd_list):.3f}\t{np.mean(rmse_list):.3f}")


In [25]:
# Model Predict
model = torch.load(os.path.join(model_path,"best_GAB.pth"),map_location=device)
model = model.eval()

psnr_list = []
ssim_list = []
gmsd_list = []
rmse_list = []
for img_ld,img_fd in tqdm(data_loader):
    img_ld, img_fd = img_ld.to(device), img_fd.to(device)
    img_ld = model(img_ld).detach()

    img_ld, img_fd = img_ld.cpu(), img_fd.cpu()
    psnr_list.append(psnr(img_ld*255,img_fd*255).item())
    ssim_list.append(ssim(img_ld*255,img_fd*255).item())
    gmsd_list.append(gmsd(img_ld*255,img_fd*255).item())
    rmse_list.append(rmse(img_ld*255,img_fd*255).item())
    

print(f"{np.mean(psnr_list):.3f}\t{np.mean(ssim_list):.3f}\t{np.mean(gmsd_list):.3f}\t{np.mean(rmse_list):.3f}")

100%|██████████| 200/200 [00:32<00:00,  6.20it/s]

30.339	0.973	0.033	7.924


## 可视化

In [ ]:
device = torch.device("cpu")
model = torch.load(os.path.join(model_path,"best_GAB.pth"),map_location=device)
model = model.eval()


img_ld,img_fd = dataset.__getitem__(180)

img_ld, img_fd = img_ld.to(device), img_fd.to(device)
pred = model(img_ld.unsqueeze(0))

pred = pred[0].detach() 
img_ld, img_fd, pred = img_ld.cpu(), img_fd.cpu(), pred.cpu()
img_fd = img_fd[0].numpy()
img_ld = img_ld[0].numpy()
ld_fd = np.abs(img_ld-img_fd)
pred = pred[0].numpy()
pred_fd = np.abs(pred-img_fd)
pred_ld = np.abs(pred-img_ld)

x,y,h,w = 250,412,100,100
imshow([img_fd,pred,pred_fd],(x,y,w,h))